In [10]:
import ccxt
import pandas as pd
import ta

# Initialize Binance exchange API
api_key = 'your api key'
api_secret = 'your api secret'
exchange = ccxt.binance({
    'apiKey': api_key,
    'secret': api_secret
})

# Define parameters for Bollinger Bands
bb_period = 20
bb_std = 2
sma_period = 5

# Get historical data for ETH/USDT trading pair
ohlcv = exchange.fetch_ohlcv('ETH/USDT', '1h')
df = pd.DataFrame(ohlcv, columns=['timestamp', 'open', 'high', 'low', 'close', 'volume'])
df['timestamp'] = pd.to_datetime(df['timestamp'], unit='ms')
df.set_index('timestamp', inplace=True)
df = df['2023-05-08':'2023-05-09']

# Calculate Bollinger Bands and SMA using ta library
bb = ta.volatility.BollingerBands(close=df['close'], window=bb_period, window_dev=bb_std)
df['bb_upperband'] = bb.bollinger_hband()
df['bb_middleband'] = bb.bollinger_mavg()
df['bb_lowerband'] = bb.bollinger_lband()
df['sma5'] = ta.trend.SMAIndicator(close=df['close'], window=sma_period).sma_indicator()


# Implement modified trading strategy
df['position'] = 0
df.loc[df['close'] > df['bb_upperband'], 'position'] = 1
df.loc[df['close'] < df['sma5'], 'position'] = -1
df['position'] = df['position'].shift()

# Calculate daily returns and cumulative returns
df['returns'] = df['close'].pct_change() * df['position']
df['cum_returns'] = (1 + df['returns']).cumprod()

# Print performance metrics
print('Total return:', round(df['cum_returns'][-1] - 1, 4))
print('Sharpe ratio:', round(df['returns'].mean() / df['returns'].std() * 252**0.5, 4))
print('Max drawdown:', round((df['cum_returns'] / df['cum_returns'].cummax() - 1).min(), 4))


Total return: -0.0151
Sharpe ratio: -1.4867
Max drawdown: -0.0244
